# SQL query from table names - Continued

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [2]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [3]:
context = [ {'role':'system', 'content':"""
 CREATE SEVERAL (3+) TABLES HERE
"""} ]



In [4]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
WRITE IN YOUR CONTEXT QUERIES HERE
"""
})

In [6]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [ ]:
#new
context_user = context.copy()
print(return_CCRMSQL("""What are the studies of the top highest paid employees?""", context_user))

```sql
SELECT e.employee_id, e.employee_name, s.study_name
FROM employees e
JOIN employee_studies es ON e.employee_id = es.employee_id
JOIN studies s ON es.study_id = s.study_id
WHERE e.salary = (SELECT MAX(salary) FROM employees);
```


In [9]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("What are the studies of the top highest paid employees?", old_context_user))

This is your SQL:
```sql
SELECT e.name, s.salary, st.educational_level, st.Institution, st.Years, st.Speciality
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
JOIN studies st ON e.ID_usr = st.ID_usr
ORDER BY s.salary DESC
LIMIT 5;
```

This SQL query retrieves the names, salaries, educational levels, institutions, years, and specialties of the top 5 highest paid employees by joining the employees, salary, and studies tables and ordering the results by salary in descending order.


In [10]:
#new
print(return_CCRMSQL("which is the location with the highest salaries per department?", context_user))

```sql
SELECT department, location, MAX(salary) AS highest_salary
FROM employees
GROUP BY department;
```


In [11]:
#old
print(return_CCRMSQL("which are the location iwth the most senior employees?", old_context_user))

This is your SQL:
```sql
SELECT e.name, MAX(salary.year) AS most_senior_year
FROM employees e
JOIN salary ON e.ID_usr = salary.ID_usr
GROUP BY e.name
ORDER BY most_senior_year DESC;
```

This SQL query retrieves the names of employees along with the year they started working (most senior employees) by joining the "employees" and "salary" tables on the employee ID. It groups the results by employee name and orders them by the most recent year to identify the location with the most senior employees.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In [14]:
# Bank example
bank_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

bank_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "customers",
  "fields": [
    {"nombre": "customer_id", "tipo": "int"},
    {"nombre": "full_name", "tipo": "varchar"},
    {"nombre": "email", "tipo": "varchar"},
    {"nombre": "phone", "tipo": "varchar"},
    {"nombre": "created_at", "tipo": "datetime"}
  ]
}
"""
})

bank_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "accounts",
  "fields": [
    {"nombre": "account_id", "type": "int"},
    {"name": "customer_id", "type": "int"},
    {"name": "account_type", "type": "varchar"},
    {"name": "currency", "type": "varchar"},
    {"name": "balance", "type": "decimal"}
  ]
}
"""
})

bank_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "transactions",
  "fields": [
    {"name": "tx_id", "type": "int"},
    {"name": "account_id", "type": "int"},
    {"name": "tx_type", "type": "varchar"},
    {"name": "amount", "type": "decimal"},
    {"name": "tx_date", "type": "datetime"},
    {"name": "merchant", "type": "varchar"},
    {"name": "description", "type": "varchar"}
  ]
}
"""
})

In [15]:
bank_context_user = bank_context.copy()
print(return_CCRMSQL("Which transaction could be a fraud and who onws them?", bank_context_user))

This is your SQL:
```sql
SELECT t.tx_id, t.tx_type, t.amount, t.merchant, c.full_name
FROM transactions t
JOIN accounts a ON t.account_id = a.account_id
JOIN customers c ON a.customer_id = c.customer_id
WHERE t.amount > 1000
```

This SQL query retrieves transactions with an amount over 1000, showing the transaction ID, type, amount, merchant, and the full name of the customer who owns the account associated with the transaction. Transactions with high amounts could be flagged as potential fraud.


In [16]:
# Store example
store_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

store_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "products",
  "fields": [
    {"nombre": "product_id", "tipo": "int"},
    {"nombre": "sku", "tipo": "varchar"},
    {"nombre": "name", "tipo": "varchar"},
    {"nombre": "category", "tipo": "varchar"},
    {"nombre": "price", "tipo": "decimal"}
  ]
}
"""
})

store_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "orders",
  "fields": [
    {"nombre": "order_id", "type": "int"},
    {"name": "customer_id", "type": "int"},
    {"name": "order_date", "type": "datetime"},
    {"name": "status", "type": "varchar"},
    {"name": "total_amount", "type": "decimal"}
  ]
}
"""
})

store_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "order_items",
  "fields": [
    {"name": "order_item_id", "type": "int"},
    {"name": "order_id", "type": "int"},
    {"name": "product_id", "type": "int"},
    {"name": "quantity", "type": "int"},
    {"name": "unit_price", "type": "decimal"},
    {"name": "discount", "type": "decimal"}
  ]
}
"""
})

In [17]:
store_context_user = store_context.copy()
print(return_CCRMSQL("What are the top 3 sellers and in which areas??", store_context_user))

This is your SQL:
```sql
SELECT p.name AS product_name, p.category AS product_category, SUM(oi.quantity) AS total_quantity
FROM products p
JOIN order_items oi ON p.product_id = oi.product_id
GROUP BY p.name, p.category
ORDER BY total_quantity DESC
LIMIT 3;
```

This SQL query retrieves the top 3 selling products along with their categories. It joins the "products" table with the "order_items" table on the product_id, calculates the total quantity sold for each product, groups the results by product name and category, orders them in descending order by total quantity, and limits the output to the top 3 results.


In [19]:
# Shakira fan club example
fanclub_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

fanclub_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "members",
  "fields": [
    {"nombre": "member_id", "tipo": "int"},
    {"nombre": "full_name", "tipo": "varchar"},
    {"nombre": "country", "tipo": "varchar"},
    {"nombre": "membership_level", "tipo": "varchar"},
    {"nombre": "joined_at", "tipo": "datetime"}
  ]
}
"""
})

fanclub_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "events",
  "fields": [
    {"nombre": "event_id", "type": "int"},
    {"name": "event_name", "type": "varchar"},
    {"name": "event_date", "type": "datetime"},
    {"name": "city", "type": "varchar"},
    {"name": "capacity", "type": "int"}
  ]
}
"""
})

fanclub_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "donations",
  "fields": [
    {"name": "donation_id", "type": "int"},
    {"name": "member_id", "type": "int"},
    {"name": "amount", "type": "decimal"},
    {"name": "donation_date", "type": "datetime"},
    {"name": "campaign", "type": "varchar"}
  ]
}
"""
})

In [20]:
fanclub_context_user = fanclub_context.copy()
print(return_CCRMSQL("Which fans have attended all of the events?", fanclub_context_user))

This is your SQL:
```sql
SELECT m.full_name
FROM members m
WHERE NOT EXISTS (
    SELECT *
    FROM events e
    WHERE NOT EXISTS (
        SELECT *
        FROM donations d
        WHERE d.member_id = m.member_id
        AND d.donation_date >= e.event_date
    )
);
```

This SQL query retrieves the full names of fans who have attended all events by checking for members who have made donations for each event's date.


Observations:
-some outputs matched the defined schemas well, for example the query using employees, salary, and studies with columns ID_usr, name, educational_level, Institution, Years, Speciality. The fan club example also stayed within members, events, and donations using valid fields like member_id, donation_date, and full_name.
-A few variations did not work. One query invented non-existent elements such as employee_id, employee_name, study_name, an employee_studies bridge table, and a salary column inside employees.
-Clear, explicit schemas and consistent field names reduce hallucinations. Adding a rule to forbid new tables or columns, plus showing one valid join pattern per scenario, improves reliability and keeps generated SQL aligned with your definitions.